This tutorial assumes you have gone through the Trace and Wavelength Calibration tutorials

Repeat the trace-and-extract process derived in trace:

In [ ]:
from PIL import Image
import numpy as np
import pylab as pl
pl.style.use('dark_background')

In [ ]:
image_array_2 = np.array(Image.open('deneb_3s_13.63g_1.bmp'))

In [ ]:
from astropy.modeling.polynomial import Polynomial1D
from astropy.modeling.fitting import LinearLSQFitter
linfitter = LinearLSQFitter()

In [ ]:
yaxis2 = np.repeat(np.arange(470, 520)[:,None], image_array_2.shape[1], axis=1)
xvals = np.arange(image_array_2.shape[1])
weighted_yaxis_values2 = np.average(yaxis2, axis=0, weights=image_array_2[470:520,:] - np.median(image_array_2))
polymodel2 = Polynomial1D(degree=3)
fitted_polymodel2 = linfitter(polymodel2, xvals, weighted_yaxis_values2)
trace_center2 = fitted_polymodel2(xvals)

In [ ]:
npixels_to_cut = 15
trace_center = fitted_polymodel2(xvals)
cutouts = np.array([image_array_2[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii]
                    for yval, ii in zip(trace_center, xvals)])
cutouts.shape
mean_trace_profile = cutouts.mean(axis=0)

In [ ]:
spectrum2 = np.array([np.average(image_array_2[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii],
                                weights=mean_trace_profile)
                     for yval, ii in zip(trace_center2, xvals)])

Retrieve the wavelength solution\\

In [ ]:
from astropy.modeling.models import Linear1D
wlmodel = Linear1D(slope=-0.10213643, intercept=562.3862495)

In [ ]:
from astropy import units as u
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
wavelengths = wlmodel(xvals) * u.nm

In [ ]:
pl.plot(wavelengths, spectrum2)

In [ ]:
from astropy.modeling.models import Gaussian1D
from astropy.modeling.fitting import LevMarLSQFitter

In [ ]:
pl.plot(wavelengths, spectrum2)
pl.axis([470,510,140,190])

In [ ]:
absorption_model_guess = Linear1D(slope=0, intercept=175) + Gaussian1D(amplitude=-25, mean=486, stddev=3)

In [ ]:
pl.plot(wavelengths, spectrum2)
pl.plot(wavelengths, absorption_model_guess(wavelengths.value))
pl.axis([470,510,140,190])

In [ ]:
lmfitter = LevMarLSQFitter()
selection = (wavelengths > 470*u.nm) & (wavelengths < 500*u.nm) 
fitted_absorption_model = lmfitter(model=absorption_model_guess, x=wavelengths.value[selection], y=spectrum2[selection])

In [ ]:
pl.plot(wavelengths, spectrum2)
pl.plot(wavelengths, fitted_absorption_model(wavelengths.value))
pl.axis([470,510,140,190])

In [ ]:
continuum_fit, absorption_fit = fitted_absorption_model

In [ ]:
pl.plot(wavelengths, spectrum2)
pl.plot(wavelengths, spectrum2 - absorption_fit(wavelengths.value))
pl.axis([470,510,140,190])

In [ ]:
EQW =  -absorption_fit(wavelengths.value[selection]).sum() / continuum_fit.intercept *u.nm
EQW

In [ ]:
absorption_fit

In [ ]:
air_wavelength_hbeta = 486.135*u.nm # wikipedia https://en.wikipedia.org/wiki/Balmer_series

In [ ]:
from astropy import constants

In [ ]:
doppler_velocity = (absorption_fit.mean*u.nm - air_wavelength_hbeta) / (air_wavelength_hbeta) * constants.c
doppler_velocity.to(u.km/u.s)

In [ ]:
doppler_velocity = (absorption_fit.mean*u.nm).to(u.km/u.s, u.doppler_optical(air_wavelength_hbeta))
doppler_velocity

In [ ]:
linewidth_kms = (absorption_fit.stddev*u.nm) / air_wavelength_hbeta * constants.c.to(u.km/u.s)
linewidth_kms

In [ ]:
pwd